In [ ]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk
import numpy as np
import tensorflow as tf
from gensim.models import KeyedVectors
import tensorflow_datasets as tfds
from keras.layers import Input, Dense, Dropout, Activation, Flatten, MultiHeadAttention, BatchNormalization, Softmax
from keras.models import Model
import time
from keras.losses import BinaryCrossentropy
from keras.utils import plot_model

In [ ]:
files = os.listdir("/content/drive/MyDrive/To Delete/chats")

In [ ]:
file_lengths = []

for file in files:
    with open("/content/drive/MyDrive/To Delete/chats/" + file, "r") as f:
        file_lengths.append(len(f.read()))

In [ ]:
plt.hist(file_lengths)

Creating a single chat source.

In [ ]:
with open("main.txt", "w") as m:
    m.write("")

for file in files:
    with open("/content/drive/MyDrive/To Delete/chats/" + file, "r") as f:
        with open("main.txt", "a") as m:
            m.write(f.read())

In [ ]:
texts_lined = []

with open("main.txt", "r") as m:
    texts_lined = m.readlines()

In [ ]:
len(texts_lined)

Data Cleaning

In [ ]:
def merge_each_msg_to_chat(texts):
    i = 0
    ans = []

    while(i < len(texts)):
        if(texts[i].startswith("[")):
            txt = texts[i]
            i += 1
            while((i < len(texts)) and (not str(texts[i]).startswith("["))):
                txt += " " + texts[i]
                i += 1
            ans.append(txt)

    return ans

In [ ]:
chats_merged = merge_each_msg_to_chat(texts_lined)

print("Percentage of data merged", len(chats_merged)/len(texts_lined))

For handling continous messages

In [ ]:
from time import strptime, mktime
from datetime import timedelta

def chat_merger(texts, time_desired):
    ans = []

    i = 0
    removed_chars = 0

    while(i<len(texts)):
        try:
            name = texts[i][20:].split(":")[0].strip()
            loop_text = texts[i]
            loop_time = mktime(strptime(texts[i][0:20], "[%d/%m/%y, %H:%M:%S]"))

            i += 1

            while((i<len(texts)) and (name == texts[i][20:].split(":")[0].strip())):
                time_new = mktime(strptime(texts[i][0:20], "[%d/%m/%y, %H:%M:%S]"))

                if((time_new - loop_time) <= 60*time_desired):
                    loop_text += " " + texts[i][20:].split(":")[1].strip()
                else:
                    break

                i += 1
                loop_time = time_new

            ans.append(loop_text)


        except:
            removed_chars += 1
            i += 1


    print("Removed", removed_chars, "invalid whatsapp texts")
    return ans

In [ ]:
merged_chats = chat_merger(chats_merged, 10)

In [ ]:
merged_chats[93]

In [ ]:
print("Percentage of data retained after merging", len(merged_chats)/len(texts_lined))

In [ ]:
print(len(texts_lined))
print(len(merged_chats))

In [ ]:
!pip install emoji
import emoji

def remove_only_emoji_text(text):
    if(emoji.emoji_count(text) == len(text[20:].split(":")[1].strip())):
        return False

    return True

In [ ]:
texts_emojis_removed = []

for texts in tqdm(merged_chats):
    if(remove_only_emoji_text(texts)):
        texts_emojis_removed.append(texts)

In [ ]:
print("Percentage of chat messages retained", len(texts_emojis_removed)/len(texts_lined))

In [ ]:
import seaborn as sns

lengths = [len(x[20:].split(":")[1].strip()) for x in texts_emojis_removed]
sns.boxplot(lengths)

In [ ]:
sns.kdeplot(lengths)

In [ ]:
sns.ecdfplot(lengths[0:int(0.99*len(lengths))])

In [ ]:
import math
import numpy as np

lengths = sorted(lengths)
for i in np.arange(0.90, 0.99, 0.01):
    print(i, "th percentile of the data is ", lengths[math.floor(i*len(lengths))])

In [ ]:
import re

def remove_weirdos(texts):
    preprocessed_text = []

    print("Removing Weirdos from the text")
    for text in texts:
        msg = str(text[20:].split(":")[1].strip().lower())

        invalid_ct = ["ommitted", "Messages and calls are end-to-end encrypted", "https", "http", "Missed voice call", "Missed video call", "deleted", "created group", ".tech", ".com", ".in"]

        if(len(msg) > 80 or len(msg) < 2):
            continue

        temp = False

        for i in invalid_ct:
            if(msg.count(i)):
                temp = True
                break

        if(temp):
            continue

        if(re.search(r"@\d+", msg)):
            continue

        regex=r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        if(re.search(regex, msg)):
            continue

        regex = r"/^([a-zA-Z\d\-\•])+@([\w-])+\•([a-z])+(\•([a-z])+)?$"
        if(re.search(regex, msg)):
            continue

        msg = (msg.encode('ascii', 'ignore')).decode('utf-8').strip()

        preprocessed_text.append(msg.replace("\n", ""))



    print("Data Retained", (len(preprocessed_text) / len(texts)) * 100, "%")

    return preprocessed_text

In [ ]:
data_preprocessed = remove_weirdos(texts_emojis_removed)

In [ ]:
words_tokenized = []

for sentences in data_preprocessed:
    words_tokenized.extend(sentences.split(" "))

In [ ]:
sorted_words_tokenized = sorted(words_tokenized)

In [ ]:
sorted_words_tokenized

In [ ]:
sorted_words_tokenized_preprocessed = []
special_chars = ['*', '?', '%', "`", "\\",  '&', "_", "<", ">","[","]", '$', '(', ')', '#', '^', '@', '!', '~', '-', '+', '=', " ", ",", "'", '"',"/", ".", ":"]

for words in tqdm(sorted_words_tokenized):
    # This can be just made for a single word in future, because for tokenizing the values not all are required.
    if(len(words) < 2):
        continue

    check_isalum = False

    for char in words:
        if char in special_chars or ord(char) > 127:
            check_isalum = True
            break

    if(check_isalum):
        continue

    if(re.search("[0-9]", words) and re.search("[a-zA-Z]", words)):
        continue

    if(re.match("^[0-9]", words)):
        continue

    if(emoji.emoji_count(words) == len(words)):
        continue

    if(len(set(words)) == 1):
        continue

    sorted_words_tokenized_preprocessed.append(words.lower())

In [ ]:
def token_checker(words):
    # Tells if it is a invalid token or not
    words = words.strip().lower()

    for char in words:
        if (char in special_chars or ord(char) > 127):
            words = words.replace(char, "")
            break

    if(re.search("[0-9]", words) and re.search("[a-z]", words)):
        return (True, "")

    if(re.match("^[0-9]", words)):
        return (True, "")

    if(emoji.emoji_count(words) == len(words)):
        return (True, "")

    if(len(words) < 2):
        return (True, "")

    if(len(set(words)) == 1):
        return (True, "")

    return (False, words)

In [ ]:
uniq_sorted = list((sorted(set(sorted_words_tokenized_preprocessed))))

print(len(uniq_sorted))

In [ ]:
mapper_english = {}
mapper_hindi = {}

In [ ]:
def word_enders(word):
    test = word[-1]
    first_test = False

    for i in word[-3:]:
        if(i!=test):
            first_test = True
            break

    if(not first_test):
        i = len(word) - 1
        while(i > 0 and word[i] == test):
            i -= 1

        mapper_hindi[word] = word[0:i+2]
        return word[0:i+2]

    else:
        return word

In [ ]:
uniq_sorted = [word_enders(word) for word in uniq_sorted]

print(len(uniq_sorted))

In [ ]:
with open("tokens_before.txt", "a") as f:
    for w in uniq_sorted:
        f.write(w + "\n")

In [ ]:
def LCS_Match(first_str, second_str):
    if(first_str[0:-1] == second_str[0:-1]):
        if("".join(sorted(first_str[-1] + second_str[-1])) == "ae"):
            mapper_hindi[second_str] = first_str
            return first_str

        if("".join(sorted(first_str[-1] + second_str[-1])) == "ai"):
            mapper_hindi[second_str] = first_str
            return first_str

        if("".join(sorted(first_str[-1] + second_str[-1])) == "ie"):
            mapper_hindi[second_str] = first_str
            return first_str

    mini_str = first_str if len(first_str) < len(second_str) else second_str
    other_str = second_str if len(first_str) < len(second_str) else first_str

    if(abs(len(mini_str) - len(other_str))) == 1:
        return mini_str

    dp = [[0]*(len(other_str)+1) for i in range(len(mini_str)+1)]



    for i in range(len(mini_str) - 1, -1, -1):
        for j in range(len(other_str) - 1, -1, -1):
            if(mini_str[i] == other_str[j]):
                dp[i][j] = 1 + dp[i+1][j+1]
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j+1])

    not_matched = ""
    p = ""

    normalizer = 1

    if(len(mini_str) > 10):
        normalizer = range(len(mini_str) - 2, len(mini_str) + 2)
    elif(len(mini_str) > 5):
        normalizer = range(len(mini_str) - 1, len(mini_str) + 1)
    else:
        normalizer = [len(mini_str)]


    if(dp[0][0] in normalizer):
        i = 0
        j = 0

        indexes = []

        while((i<len(dp)) and (j<len(dp[0]))):
            while((j+1<len(dp[0])) and dp[i][j+1] == dp[i][j]):
                indexes.append(j)
                j += 1

            i += 1
            j += 1

        for i in range(0, len(other_str)):
            if(i in indexes):
                if(i+1 >= len(other_str)):
                    p += other_str[i]
                elif(other_str[i] != other_str[i+1]):
                    p += other_str[i]
            else:
                p += other_str[i]

        diff_length = abs(len(mini_str) - len(p))

        if(diff_length < 2):
            mapper_hindi[first_str] = p
            mapper_hindi[second_str] = p
            return p
        else:
            return ""

    else:
        return p

In [ ]:
LCS_Match("ab", "abe")

If this matches with the bigger string, it is not relevant to remove it. Also, if it returns none, means that the matching is not in proper range, so not feasible to merge/map the words.

In [ ]:
len(uniq_sorted)

In [ ]:
def preprocess_epoch(data):
    len_final_tokens = len(data)

    i = 0
    final_tokens = []

    while(i+1 < len(data)):
        result = LCS_Match(data[i], data[i+1])
        if(len(result) > 0):
            i += 2
            final_tokens.append(result)
        else:
            final_tokens.append(data[i])
            i += 1


    final_tokens = [word_enders(word) for word in final_tokens]
    print("Data retained", len(final_tokens)/14033)

    return final_tokens

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('words')
nltk.download('wordnet')
from nltk.corpus import words as english_dictionary
lemmatizer = WordNetLemmatizer()


vocab = set(english_dictionary.words())

english_words = []

for word in tqdm(uniq_sorted):
    english_words.append(lemmatizer.lemmatize(word))

for i, word in enumerate(english_words):
    if(word in vocab):
        mapper_english[uniq_sorted[i]] = word

In [ ]:
mapper_created_data = uniq_sorted
plot = []

for i in range(0, 6):
    mapper_created_data = preprocess_epoch(mapper_created_data)
    plot.append(len(mapper_created_data) / 14033)

In [ ]:
with open("tokens.txt", "w") as f:
    # for word in data:
    #     f.write(word + "\n")
    for i in mapper_hindi.items():
        f.write(i[0] + " : " + i[1] + "\n")

In [ ]:
from collections import defaultdict
messages_improved = []

def fact():
    return 0

tokens_improved = defaultdict(fact)
from nltk.corpus import words

for i in data_preprocessed:
    data = i.split(" ")
    sentence = ""

    for j in data:
        check, result = token_checker(j)
        # check tells if the token is invalid
        if(check):
            continue

        if(result in mapper_english.keys()):
            sentence += mapper_english[result] + " "
            tokens_improved[str(mapper_english[result])] += 1
        elif(result in mapper_hindi.keys()):
            sentence += (str(mapper_hindi[result] + " "))
            tokens_improved[str(mapper_hindi[result])] += 1
        else:
            sentence += (result + " ")
            tokens_improved[result] += 1

    sentence = sentence.strip()
    messages_improved.append(sentence)

In [ ]:
import pandas as pd

tokens = pd.DataFrame({
    "token_string": tokens_improved.keys(),
    "token_count": tokens_improved.values(),
})

In [ ]:
tokens['token_count'].value_counts().plot()

In [ ]:
less_occuring_tokens = set(tokens[tokens['token_count'] <= 4]['token_string'].values)

In [ ]:
messages_final = []

for msg in tqdm(messages_improved):
    if(len(msg) == 0):
        continue

    temp = msg.split(" ")
    intersected = less_occuring_tokens.intersection(temp)

    text_str = ""

    if(len(intersected) > 0):
        for m in temp:
            if(m not in intersected):
                text_str += m + " "
            else:
                text_str += "<UNK> "
    else:
        text_str = msg

    messages_final.append(text_str.strip())

In [ ]:
maxl = 0

for i, val in enumerate(messages_final):
    messages_final[i] = val.split(" ")
    maxl = np.max([len(messages_final[i]), maxl])

In [ ]:
import gensim
from gensim.models import Word2Vec

model = gensim.models.Word2Vec(sentences = messages_final, vector_size=128, min_count = 1, window = 5, sg=0)
# Generates One Hot Encoding

In [ ]:
model.train(messages_final, epochs=100,total_examples = len(messages_final))
model.save("word2vec_embeddings.model")

In [ ]:
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [ ]:
embeddings = KeyedVectors.load("word2vec.wordvectors")
key_to_embedding = dict(zip(embeddings.index_to_key, embeddings.vectors))

In [ ]:
EMBEDDING_LENGTH = 128
SEQ_LENGTH = 4
BUFFER_SIZE = 3000
BATCH_SIZE = 1
NUM_HEADS = 8
KEY_DIM = 64 #paper
MHADROPOUT = 0.2
WINDOW_SIZE = 4
MAXL = 21
EPOCHS = 20
OPTIMIZER_LR = 0.0001

In [ ]:
def embed_sentence_with_masking(seq):
    values = []
    for i in seq:
        values.append(key_to_embedding[i])

    return tf.convert_to_tensor(values)

def positional_encoder(pos = SEQ_LENGTH, dmodel = EMBEDDING_LENGTH):
    encoder = []

    for p in range(pos):
        temp = []
        for i in range(0, dmodel):
            if(i % 2 == 0):
                temp.append(np.sin(p / np.power(10000, (2*i) // dmodel)))
            else:
                temp.append(np.cos(p / np.power(10000, (2*i) // dmodel)))

        encoder.append(temp)

    return np.array(encoder)

In [ ]:
positional_embeddings = positional_encoder()

plt.pcolormesh(positional_embeddings, cmap='RdBu')
plt.xlabel('Depth')
plt.ylabel('Position')
plt.colorbar()
plt.show()

In [ ]:
look_ahead_masks = []

for i in range(0, SEQ_LENGTH-1):
    look_ahead_masks.append([1]*(i+1) + [0] * (SEQ_LENGTH-1-i))

len(look_ahead_masks)

In [ ]:
temp = []
unique_word_set = dict()
int_to_word_set = dict()

ptr = 0

for i in messages_final:
    temp.extend(i)

    for j in i:
        if j not in unique_word_set.keys():
            unique_word_set[j] = ptr
            int_to_word_set[ptr] = j
            ptr += 1

temp = np.array(temp)

In [ ]:
vocab_length = len(unique_word_set)

In [ ]:
x_tensor = []
y_tensor = []

for i, val in enumerate(temp):
    if(i+WINDOW_SIZE < len(temp)):
        x_tensor.append(temp[i:i+WINDOW_SIZE])
        y_tensor.append(temp[i+WINDOW_SIZE])

x_tensor = tf.convert_to_tensor(np.array(x_tensor))
y_tensor = tf.convert_to_tensor(np.array(y_tensor))

In [ ]:
data = tf.data.Dataset.from_tensor_slices((x_tensor, y_tensor))

test_data = data.take(int(0.1*len(data)))
train_data = data.skip(int(0.1*len(data)))

In [ ]:
print(len(test_data))
print(len(train_data))

In [ ]:
def masked_binary_tensor(seq):
    return tf.cast(tf.equal(seq, 0), dtype=tf.float32)

In [ ]:
def make_batches(ds):
    return ds.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
test_data = make_batches(test_data)
train_data = make_batches(train_data)

In [ ]:
def create_look_ahead_mask():
    vec = 1 - tf.linalg.band_part(tf.ones((SEQ_LENGTH,SEQ_LENGTH)), -1, 0)
    return vec

200 sized embedding -> add with positional_embeddings -> iterate it 8 times ->  Predict the word, penalize with cross entropy (Ignore the ones with padding) -> Train decoder

In [ ]:
class WhatsGPT(tf.keras.Model):
    def __init__(self, num_heads, key_dim, value_dim, dropout, ff_layers, vocab_length):
        super(WhatsGPT, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.value_dim = value_dim
        self.dropout = dropout
        self.ff_layers = ff_layers

        self.mha = MultiHeadAttention(num_heads, key_dim, value_dim, dropout)
        self.ff = Dense(ff_layers, activation='relu')

        self.bn_1 = BatchNormalization()
        self.bn_2 = BatchNormalization()

        self.dense = Dense(vocab_length, activation='linear')

        self.probabs = Softmax()

    def call(self, train_data, look_ahead_mask, training):
        attention_output = self.mha(train_data, train_data, train_data, look_ahead_mask, training=training) # (batch, seq_length, emb_size)

        add_and_norm_1 = attention_output + train_data # (batch, seq_length, emb_size)

        batch_normalized = self.bn_1(add_and_norm_1) # (batch, seq_length, emb_size)

        feed_forward_output = self.ff(add_and_norm_1) # (batch, seq_length, emb_size)

        add_and_norm_2 = batch_normalized + feed_forward_output # (batch, seq_length, emb_size)

        batch_normalized_2 = self.bn_2(add_and_norm_2) # (batch, seq_length, emb_size)

        final_output = self.dense(batch_normalized_2) # (batch, seq_length, emb_size)

        final_output_probabs = self.probabs(final_output) # (batch, seq_length, emb_size)

        return final_output_probabs

In [ ]:
look_ahead_mask = create_look_ahead_mask()

In [ ]:
whatsgpt = WhatsGPT(NUM_HEADS, KEY_DIM, KEY_DIM, MHADROPOUT, ff_layers=EMBEDDING_LENGTH, vocab_length=len(unique_word_set))

In [ ]:
loss = BinaryCrossentropy(reduction="none")
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [ ]:
def loss_function(pred, real):
    return loss(real, pred)

optimizer = tf.keras.optimizers.legacy.Adam(OPTIMIZER_LR, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
def accuracy_calculator(real, pred):
    accuracies = tf.cast(tf.equal(real, tf.cast(tf.argmax(pred, axis=2), tf.float32)), tf.int32)
    return tf.reduce_sum(accuracies)

In [ ]:
def create_one_hot_encoded_vectors(target):
    ohe_vectors = []

    for i in target:
        ohe_vectors.append(tf.one_hot(unique_word_set[i], vocab_length))

    return tf.convert_to_tensor(ohe_vectors, dtype=tf.float32)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/To Delete/whatsGPT LLM Checkpoints"

ckpt = tf.train.Checkpoint(whatsgpt)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [ ]:
for epoch in range(EPOCHS):
    print("EPOCH: ", epoch)
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    ckpt_save_path = ckpt_manager.save()
    print (f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')

    for (batch, (inp, tar)) in enumerate(train_data):
        embeddings = inp.numpy().astype(type(''))[0]
        target = np.concatenate((inp.numpy().astype(type(''))[0][1:], tar.numpy().astype(type(""))))

        embeddings = embed_sentence_with_masking(embeddings)
        embeddings += positional_embeddings

        final_vector = create_one_hot_encoded_vectors(target)

        embeddings = embeddings[tf.newaxis, :, :]

        with tf.GradientTape() as tape:
            predictions = whatsgpt(embeddings, look_ahead_mask, True)

            predictions = predictions[tf.newaxis, :, :]
            loss_2 = loss_function(tf.transpose(predictions), final_vector)
            gradients = tape.gradient(loss_2, whatsgpt.trainable_weights)

            optimizer.apply_gradients(zip(gradients, whatsgpt.trainable_weights))

            train_loss(loss_2)
            train_accuracy(accuracy_calculator(final_vector, predictions)/WINDOW_SIZE)


        if (batch + 1) % 50 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

    end = time.time()
    print("Time taken to complete:", end - start)

In [ ]:
whatsgpt.save_weights("model_tech_community_10.h5")

In [ ]:
text = ["i", "love", "github", "do", "you"]
# TODO: Preprocess before entering into the prediction function

i = len(text)

while(i<12):
    if len(text) > 4:
      text = text[:-4]

    embed = embed_sentence_with_masking(text)

    embed = embed[tf.newaxis, :,:]

    output = whatsgpt(embed, None, False)

    text.append(int_to_word_set[tf.argmax(output[0][0]).numpy()])
    print(int_to_word_set[tf.argmax(output[0][0]).numpy()])

    i += 1